# Deep Agents with LangChain

This notebook demonstrates how to create and use Deep Agents with LangChain.

Deep Agents are more sophisticated than basic ReAct agents. They:
- Can handle complex, multi-step tasks
- Maintain state across multiple interactions
- Use memory to recall previous interactions
- Can decompose complex problems into subtasks

## What You'll Learn:
- Building agents with memory
- Creating agent chains for complex workflows
- Using conversation history
- Implementing reflection and self-correction

## 1. Setup and Imports

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain import hub

# Import our custom tools
from tools.tavily_search import get_tavily_search_tool

# Load environment variables
load_dotenv()

print("✓ Imports successful!")

## 2. Set Up API Keys

In [ ]:
# Uncomment and set your API keys if not using .env file
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["TAVILY_API_KEY"] = "your-tavily-api-key"

# Verify keys are set
if "OPENAI_API_KEY" in os.environ and "TAVILY_API_KEY" in os.environ:
    print("✓ API keys are configured!")
else:
    print("⚠ Warning: Please set your API keys")

## 3. Initialize Components

For deep agents, we'll use:
- A more advanced LLM (GPT-4)
- Memory to maintain conversation context
- Tools for external interactions

In [ ]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7,  # Slightly higher temperature for more creative responses
)

# Get tools
search_tool = get_tavily_search_tool(max_results=5, search_depth="advanced")
tools = [search_tool]

print(f"✓ LLM initialized: {llm.model_name}")
print(f"✓ Tools configured: {[tool.name for tool in tools]}")

## 4. Create Agent with Memory

Memory allows the agent to remember previous interactions and maintain context.

In [ ]:
# Create conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="output"
)

# Create a custom prompt with memory placeholder
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are a helpful AI assistant with access to web search.
    You can search for information, answer questions, and help with various tasks.
    Use the available tools when needed to provide accurate, up-to-date information.
    
    When solving complex problems:
    1. Break them down into smaller steps
    2. Search for information as needed
    3. Synthesize findings into a coherent answer
    4. Reflect on whether your answer fully addresses the question
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create the agent with memory
agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create agent executor with memory
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=15  # Allow more iterations for complex tasks
)

print("✓ Deep agent with memory created!")

## 5. Example 1: Multi-Turn Conversation

The agent can maintain context across multiple queries.

In [ ]:
# First query
result1 = agent_executor.invoke({
    "input": "What is Retrieval-Augmented Generation (RAG)?"
})

print("\n" + "="*80)
print("ANSWER 1:")
print("="*80)
print(result1["output"])

In [ ]:
# Follow-up query (uses context from previous conversation)
result2 = agent_executor.invoke({
    "input": "What are some popular frameworks for implementing it?"
})

print("\n" + "="*80)
print("ANSWER 2:")
print("="*80)
print(result2["output"])

In [ ]:
# Another follow-up
result3 = agent_executor.invoke({
    "input": "Can you give me a code example using one of those frameworks?"
})

print("\n" + "="*80)
print("ANSWER 3:")
print("="*80)
print(result3["output"])

## 6. View Conversation History

Let's examine what the agent remembers.

In [ ]:
# Check conversation history
print("Conversation History:")
print("="*80)
for message in memory.chat_memory.messages:
    if isinstance(message, HumanMessage):
        print(f"\n👤 Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"\n🤖 AI: {message.content[:200]}...")  # Show first 200 chars

## 7. Example 2: Complex Problem Decomposition

Deep agents can break down complex problems into manageable steps.

In [ ]:
# Clear memory for a fresh start
memory.clear()

# Complex query requiring multiple steps
complex_query = """I want to build a chatbot for customer service. 
Can you help me understand:
1. What technologies I should use
2. The main challenges I'll face
3. Best practices for implementation
"""

result = agent_executor.invoke({"input": complex_query})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
print(result["output"])

## 8. Example 3: Research and Synthesis

The agent can gather information from multiple sources and synthesize it.

In [ ]:
# Clear memory
memory.clear()

# Research query
research_query = """Compare LangChain, LlamaIndex, and Haystack for building RAG applications.
Which one would you recommend for a beginner and why?
"""

result = agent_executor.invoke({"input": research_query})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
print(result["output"])

## 9. Example 4: Self-Reflection and Correction

Deep agents can reflect on their answers and provide corrections if needed.

In [ ]:
# Clear memory
memory.clear()

# Initial query
result1 = agent_executor.invoke({
    "input": "What are the main components of a typical LLM application?"
})

print("\n" + "="*80)
print("INITIAL ANSWER:")
print("="*80)
print(result1["output"])

# Ask for reflection
result2 = agent_executor.invoke({
    "input": "Did you miss anything important? Please review and add any missing components."
})

print("\n" + "="*80)
print("REFLECTED ANSWER:")
print("="*80)
print(result2["output"])

## 10. Advanced: Creating a Planning Agent

Deep agents can create and execute plans for complex tasks.

In [ ]:
# Clear memory
memory.clear()

# Planning prompt
planning_query = """I want to create a blog post about 'The Future of AI in Healthcare'.
Please create a detailed plan including:
1. Key topics to research
2. Structure of the blog post
3. Important points to cover
"""

result = agent_executor.invoke({"input": planning_query})

print("\n" + "="*80)
print("PLAN:")
print("="*80)
print(result["output"])

In [ ]:
# Execute the first step of the plan
result = agent_executor.invoke({
    "input": "Now, let's execute step 1. Research the latest developments in AI for healthcare."
})

print("\n" + "="*80)
print("RESEARCH RESULTS:")
print("="*80)
print(result["output"])

## 11. Understanding Deep Agent Architecture

Deep agents differ from basic ReAct agents in several ways:

### Memory
- Maintains conversation history
- Can reference previous interactions
- Builds context over time

### Planning
- Can decompose complex tasks
- Creates multi-step plans
- Adjusts plans based on results

### Reflection
- Reviews own outputs
- Identifies gaps or errors
- Self-corrects when needed

### Tool Use
- Strategic tool selection
- Chained tool usage
- Result synthesis

## 12. Custom Query Playground

Try your own complex queries!

In [ ]:
# Clear memory for fresh start
memory.clear()

# Your custom query here
custom_query = """Help me understand how to build a production-ready RAG system.
What are the key considerations?
"""

result = agent_executor.invoke({"input": custom_query})

print("\n" + "="*80)
print("ANSWER:")
print("="*80)
print(result["output"])

## Summary

In this notebook, you learned:
- ✓ How to create deep agents with memory
- ✓ Managing multi-turn conversations
- ✓ Decomposing complex problems
- ✓ Implementing self-reflection
- ✓ Creating planning agents

## Key Differences: ReAct vs Deep Agents

| Feature | ReAct Agent | Deep Agent |
|---------|-------------|------------|
| Memory | No | Yes |
| Context | Single query | Multi-turn |
| Planning | Basic | Advanced |
| Reflection | No | Yes |
| Complexity | Simple tasks | Complex workflows |

## Next Steps
- Implement custom memory types (e.g., vector store memory)
- Add more specialized tools
- Create agent chains for workflows
- Experiment with different LLM parameters
- Build domain-specific agents